In [1]:
import pyspark
pyspark.__version__

'3.4.0'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
my_log_reg_model = LogisticRegression()
logreg_model = my_log_reg_model.fit(my_data)

In [9]:
log_summary = logreg_model.summary
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000486...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716171...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298141...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113070...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823781...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446123...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [13]:
log_summary.accuracy

1.0

In [14]:
lr_train,lr_test = my_data.randomSplit([0.7,0.3])

In [15]:
final_model = LogisticRegression()

In [16]:
fit_final = final_model.fit(lr_train)

In [17]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [18]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[123,124,125...|[40.1141984002090...|           [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[40.2285603525944...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[38.8831388194583...|           [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|[33.6097084557222...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[36.5763089729879...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[23.9686242244011...|[0.99999999996104...|       0.0|
|  0.0|(692,[126,127,128...|[35.4156149618468...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[31.4854802262040...|[0.99999999999997...|       0.0|
|  0.0|(692,[150,151,152...|[25.8338953891280...|[0.99999999999396...|       0.0|
|  0.0|(692,[153

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [21]:
my_eval = BinaryClassificationEvaluator()

In [22]:
final_roc = my_eval.evaluate(dataset=prediction_and_labels.predictions)
final_roc

1.0